# Data et Set

Nous avons vu dans le chapitre précédent qu'il est possible d'**afficher** uniquement les variables qui nous intéressent. Nous avons vu également qu'il est possible d'**afficher** que les observations qui nous intéressent. Par exemple dans un ensemble de données sur des clients, nous pouvons **afficher** seulement les noms, prénoms et âge des clients habitant à Montréal.

Toutefois, avec la procédure `Format` nous modifions seulement l'**affichage**. Si nous voulons effectuer une opération quelconque (calcul, régression...etc.) sur les variables et les observations d'intérêt, il faudrait extraire (ou dupliquer) un sous-ensemble de données.


```
DATA sous-ensemble-données;
    SET ensemble-données;
    WHERE variable_1='ce_qui_nous_interesse_1'
        and variable_2='ce_qui_nous_interesse_2'
RUN;
```

Le sous-ensemble de données peut-être temporaire avec la bibliothèque `WORK.sous_sensemble` ou dans une bibliothèque permanente `LIBNAME.sous_sensemble`.

On peut extraire les observations d'intérêt avec la déclaration `WHERE`. Nous pouvons spécifier ce qui nous intéresse avec la description **exacte** si nous la connaissons, par exemple `WHRE type_prof='Ingénieur`. Lorsque nous ne sommes pas sûrs de l’écriture exacte de ce que nous cherchons, ou par exemple nous voulons extraire tout ce qui contient les deux lettres `IN`, nous utilisons les `%...%`. Par exemple `WHRE type_prof='%In%`. Dans ce dernier cas, nous aurons tous les types de professions contenant les deux lettres `In`.

### Exemple;

Reprenons notre base de données d'assurance;

In [1]:
libname assu 'data';
Data assu.cours_2;
    set assu.eq_2;
/* nous verrons un peu plus loin à quoi sert keep     */
    keep numeropol type_prof alimentation cout_total cout1 cout2; 
run;
proc print data=assu.cours_2 (firstobs = 2 obs = 5) noobs;
run;

SAS Connection established. Subprocess id is 23973



numeropol,type_prof,alimentation,cout1,cout2
52484,Ingénieur,Carnivore,.,.
27817,Ingénieur,Carnivore,.,.
32986,Technicien,Carnivore,230.280,287.217
45769,Technicien,Carnivore,.,.


On se rappelle des types de professions que nous avions dans notre base de données originale.

<style type="text/css">
	table.tableizer-table {
		font-size: 12px;
		border: 1px solid #CCC; 
		font-family: Arial, Helvetica, sans-serif;
	} 
	.tableizer-table td {
		padding: 4px;
		margin: 3px;
		border: 1px solid #CCC;
	}
	.tableizer-table th {
		background-color: #104E8B; 
		color: #FFF;
		font-weight: bold;
	}
</style>
<table class="tableizer-table">
<thead><tr class="tableizer-firstrow"><th>type_prof</th></tr></thead><tbody>
 <tr><td>Actuaire</td></tr>
 <tr><td>Autre</td></tr>
 <tr><td>Avocat</td></tr>
 <tr><td>Hockeyeur</td></tr>
 <tr><td>Infirmière</td></tr>
 <tr><td>Informaticien</td></tr>
 <tr><td>Ingénieur</td></tr>
 <tr><td>Médecin</td></tr>
 <tr><td>Professeur</td></tr>
 <tr><td>Technicien</td></tr>
</tbody></table>

Nous voudrions créer un sous-ensemble de données **temporaire** appelé `bd_ingenieurs` sur lequel on veut travailler. Ce dernier provient de l'ensemble de données `assu.eq_2` et contient seulement les données sur les ingénieurs.

In [22]:
Data work.bd_ingenieurs;
    set assu.cours_2;
    where type_prof='Ingénieur';
run;
proc print data=work.bd_ingenieurs (firstobs = 2 obs = 10);
run;


Obs,numeropol,type_prof,alimentation,cout1,cout2
2,27817,Ingénieur,Carnivore,.,.
3,19651,Ingénieur,Végétalien,.,.
4,12507,Ingénieur,Carnivore,.,.
5,8670,Ingénieur,Carnivore,.,.
6,17867,Ingénieur,Carnivore,.,.
7,5127,Ingénieur,Carnivore,.,.
8,53543,Ingénieur,Carnivore,.,.
9,27131,Ingénieur,Carnivore,289.311,.
10,16186,Ingénieur,Carnivore,.,.


Maintenant, on veut les types de professions qui ont les deux **In** comme **Infirmière** ou **Ingénieur**

In [23]:
Data work.bd_in;
    set assu.cours_2;
    where type_prof like '%In%';
run;
proc print data=work.bd_in (firstobs = 2 obs = 10);
run;


Obs,numeropol,type_prof,alimentation,cout1,cout2
2,27817,Ingénieur,Carnivore,.,.
3,13590,Infirmière,Carnivore,509.261,2665.25
4,76422,Infirmière,Carnivore,.,.
5,20860,Infirmière,Végétarien,487.640,.
6,49010,Infirmière,Végétarien,.,.
7,30089,Informaticien,Carnivore,.,.
8,17174,Informaticien,Végétarien,458.789,.
9,38332,Infirmière,Végétalien,.,.
10,19651,Ingénieur,Végétalien,.,.


On peut vérifier que nous avons les types de proffessions avec `In`


<style type="text/css">
	table.tableizer-table {
		font-size: 12px;
		border: 1px solid #CCC; 
		font-family: Arial, Helvetica, sans-serif;
	} 
	.tableizer-table td {
		padding: 4px;
		margin: 3px;
		border: 1px solid #CCC;
	}
	.tableizer-table th {
		background-color: #104E8B; 
		color: #FFF;
		font-weight: bold;
	}
</style>
<table class="tableizer-table">
<thead><tr class="tableizer-firstrow"><th>type_prof</th></tr></thead><tbody>
 <tr><td>Infirmière</td></tr>
 <tr><td>Informaticien</td></tr>
 <tr><td>Ingénieur</td></tr>
</tbody></table>

Regardons maintenant les types de proffesions se terminant par `eur`

In [25]:
Data work.bd_eur;
    set assu.cours_2;
    where type_prof like '%eur';
run;
proc print data=work.bd_eur (firstobs = 2 obs = 10);
run;

Obs,numeropol,type_prof,alimentation,cout1,cout2
2,27817,Ingénieur,Carnivore,.,.
3,56107,Hockeyeur,Végétarien,.,.
4,19651,Ingénieur,Végétalien,.,.
5,44235,Professeur,Carnivore,.,.
6,24013,Hockeyeur,Végétarien,.,.
7,12507,Ingénieur,Carnivore,.,.
8,28741,Professeur,Carnivore,.,.
9,14731,Hockeyeur,Carnivore,67.901,1639.48
10,3977,Hockeyeur,Carnivore,263.075,.


Nous obtenons alors que: 

<style type="text/css">
	table.tableizer-table {
		font-size: 12px;
		border: 1px solid #CCC; 
		font-family: Arial, Helvetica, sans-serif;
	} 
	.tableizer-table td {
		padding: 4px;
		margin: 3px;
		border: 1px solid #CCC;
	}
	.tableizer-table th {
		background-color: #5F8B08; 
		color: #FFF;
		font-weight: bold;
	}
</style>
<table class="tableizer-table">
<thead><tr class="tableizer-firstrow"><th>type_prof</th></tr></thead><tbody>
 <tr><td>Hockeyeur</td></tr>
 <tr><td>Ingénieur</td></tr>
 <tr><td>Professeur</td></tr>
</tbody></table>

Si vous êtes curieux de voir comment nous obtenons ce résultat, voici la commande `PROC sql` que nous verrons plus en détail dans les prochains cours

In [26]:
proc sql;
    create table eur as
    select type_prof, count(*) as type_prof from work.bd_eur  
    group by type_prof;
quit;
proc print data=eur noobs;
run;

type_prof
Hockeyeur
Ingénieur
Professeur


Maintenant, essayons d'extraire un sous-ensemble de données avec les critères suivants;
1. Hokeyeur
2. Qui est végétarien ou végétalien

In [27]:
Data work.bd_hk_veg;
    set assu.cours_2;
    where type_prof='Hockeyeur'
    and alimentation like 'Vé%';
run;
proc print data=work.bd_hk_veg (firstobs = 2 obs = 10);
run;

Obs,numeropol,type_prof,alimentation,cout1,cout2
2,24013,Hockeyeur,Végétarien,.,.
3,24660,Hockeyeur,Végétarien,.,.
4,56436,Hockeyeur,Végétarien,.,.
5,18535,Hockeyeur,Végétalien,.,.
6,21672,Hockeyeur,Végétarien,.,.
7,12342,Hockeyeur,Végétarien,.,.
8,74410,Hockeyeur,Végétarien,.,.
9,15159,Hockeyeur,Végétarien,456.261,.
10,32260,Hockeyeur,Végétarien,.,.


### Création d'une nouvelle variable:

Lorsque nous produisons un sous-ensemble de données, nous pouvons aussi créer une toute nouvelle variable à partir d'une opération quelconque sur les variables existantes. Par exemple, calculer le total des coûts de sinistre.

Comme dans l'exemple précédent, pour les Hokeyeurs qui ne mangent pas de viande, calculons le total des coûts de sinistres cout1-cout7;

**Remarquez** dans le code ci-dessous que nous avons un montant à la variable `cout1`. Toutefois, le total n'est pas affiché à la neuvième observation. En effet, quand on somme avec des `+`, SAS considère les valeurs manquantes dans le calcul.

In [10]:
Data work.bd_hk_veg;
    set assu.cours_2;
    cout_total=cout1+cout2+cout3+cout4+cout5+cout6+cout7;
    where type_prof='Hockeyeur'
    and alimentation like 'Vé%'; 
    keep numeropol	type_prof	alimentation cout1 cout_total; 
run;
proc print data=work.bd_hk_veg (firstobs = 2 obs = 10) noobs;
format cout1-cout7 cout_total dollar12.2;
run;

numeropol,type_prof,alimentation,cout1,cout_total
24013,Hockeyeur,Végétarien,.,.
24660,Hockeyeur,Végétarien,.,.
56436,Hockeyeur,Végétarien,.,.
18535,Hockeyeur,Végétalien,.,.
21672,Hockeyeur,Végétarien,.,.
12342,Hockeyeur,Végétarien,.,.
74410,Hockeyeur,Végétarien,.,.
15159,Hockeyeur,Végétarien,456.261,.
32260,Hockeyeur,Végétarien,.,.


La fonction `sum` règle ce problème;

In [14]:
Data work.bd_hk_veg;
    set assu.cours_2;
    cout_total=sum(cout1,cout2, cout3, cout4, cout5, cout6, cout7);
    where type_prof='Hockeyeur'
    and alimentation like 'Vé%';
    keep numeropol	type_prof	alimentation cout_total; 
run;
proc print data=work.bd_hk_veg (firstobs = 2 obs = 10) noobs;
format cout1-cout7 cout_total dollar12.2;
run;

numeropol,type_prof,alimentation,cout_total
24013,Hockeyeur,Végétarien,.
24660,Hockeyeur,Végétarien,.
56436,Hockeyeur,Végétarien,.
18535,Hockeyeur,Végétalien,.
21672,Hockeyeur,Végétarien,.
12342,Hockeyeur,Végétarien,.
74410,Hockeyeur,Végétarien,.
15159,Hockeyeur,Végétarien,$456.26
32260,Hockeyeur,Végétarien,.


Maintenant que nous avons vu comment extraire les données, nous voudrions garder dans notre nouvelle base de données seulement les variables d’intérêt. Autrement dit, nous voulons **garder** ou **ignorer** certaines variables. 

SAS possède l'option **Keep** ou **Drop** à cet effet.

```
DATA sous-ensemble-données;
    SET ensemble-données;
    WHERE variable_1='ce_qui_nous_interesse_1'
        and variable_2='ce_qui_nous_interesse_2'
    drop variable_1 variable_6 variable_7-variable_99
RUN;
```

ou

```
DATA sous-ensemble-données;
    SET ensemble-données;
    WHERE variable_1='ce_qui_nous_interesse_1'
        and variable_2='ce_qui_nous_interesse_2'
    keep variable_2 variable_3-variable_5 variable_7-variable_99
RUN;
```

Comment choisit-on entre les deux options?

Tout dépend bien sûr des variables dont nous avons besoin et du nombre de variables que nous avons à ignorer.

### Exemple avec `keep`

In [33]:
Data work.bd_hk_veg;
    set assu.cours_2;
    cout_total=sum(cout1,cout2);
    where type_prof='Hockeyeur'
    and alimentation like 'Vé%';
    keep numeropol type_prof alimentation cout_total;
run;
proc print data=work.bd_hk_veg (firstobs = 2 obs = 10);
format cout1-cout2 cout_total dollar12.2;
run;

Obs,numeropol,type_prof,alimentation,cout_total
2,24013,Hockeyeur,Végétarien,.
3,24660,Hockeyeur,Végétarien,.
4,56436,Hockeyeur,Végétarien,.
5,18535,Hockeyeur,Végétalien,.
6,21672,Hockeyeur,Végétarien,.
7,12342,Hockeyeur,Végétarien,.
8,74410,Hockeyeur,Végétarien,.
9,15159,Hockeyeur,Végétarien,$456.26
10,32260,Hockeyeur,Végétarien,.


### Exemple avec `drop`

Si nos variables d'intérêt sont (numeropol, type_prof, alimentation, cout_total), on voit que l'option la plus courte serait d'utiliser **`keep`**

In [51]:
Data work.bd_hk_veg;
    set assu.cours_2;
    cout_total=sum(cout1,cout2);
    where type_prof='Hockeyeur'
    and alimentation like 'Vé%';
    drop debut_pol	fin_pol	freq_paiement	langue	type_territoire	utilisation	
    presence_alarme	annee_veh	marque_voiture	sexe	date_naissance	
    couleur_voiture	annee_permis	myopie	cheveux	cout1-cout7	nbsin	equipe	;
run;
proc print data=work.bd_hk_veg (firstobs = 2 obs = 10) noobs;
format cout1-cout2 cout_total dollar12.2;
run;

numeropol,type_prof,alimentation,cout_total
24013,Hockeyeur,Végétarien,.
24660,Hockeyeur,Végétarien,.
56436,Hockeyeur,Végétarien,.
18535,Hockeyeur,Végétalien,.
21672,Hockeyeur,Végétarien,.
12342,Hockeyeur,Végétarien,.
74410,Hockeyeur,Végétarien,.
15159,Hockeyeur,Végétarien,$456.26
32260,Hockeyeur,Végétarien,.


## Extraire un sous-ensemble avec de `IF`

Imaginons que nous voulons maintenant extraire un sous-ensemble de données avec les caractéristiques suivantes (Hokeyeur, végétarien ou végétalien, et que les coûts de sinistre est supérieur à 1000$).

Bien évidemment, on ne peut pas faire un `WHERE` car la variable cout_total n'existe pas dans l'ensemble de données originales. Cette dernière a été créée seulement dans le sous-ensemble de données.

La fonction `IF` sert justement  à résoudre ce genre de situation;

In [50]:
Data work.bd_hk_veg;
    set assu.cours_2;
    cout_total=sum(cout1,cout2);
    where type_prof='Hockeyeur'
    and alimentation like 'Vé%';
    keep numeropol type_prof alimentation cout_total;
    if cout_total>1000;
run;
proc print data=work.bd_hk_veg (firstobs = 2 obs = 10) noobs;
format cout1-cout2 cout_total dollar12.2;
run;

numeropol,type_prof,alimentation,cout_total
84568,Hockeyeur,Végétalien,"$1,000.81"
51085,Hockeyeur,Végétarien,"$70,040.85"
12443,Hockeyeur,Végétarien,"$1,536.96"
60675,Hockeyeur,Végétarien,"$3,105.94"
1095,Hockeyeur,Végétarien,"$22,753.50"
312,Hockeyeur,Végétarien,"$1,157.76"
312,Hockeyeur,Végétarien,"$38,509.17"
18535,Hockeyeur,Végétarien,"$3,857.07"
44485,Hockeyeur,Végétarien,"$1,106.05"


Maintenant, supposons que nous voulons afficher `coût total en CAD` au lieu de `cout_total`

On se rappelle que nous avons l'option `Label` à cet effet.

In [49]:
Data work.bd_hk_veg;
    set assu.cours_2;
    cout_total=sum(cout1,cout2);
    where type_prof='Hockeyeur'
    and alimentation like 'Vé%';
    keep numeropol type_prof alimentation cout_total;
    if cout_total>1000;
    Label cout_total='coût total en CAD';
run;
proc print data=work.bd_hk_veg (firstobs = 2 obs = 10) label noobs;
format cout1-cout2 cout_total dollar12.2;
run;

numeropol,type_prof,alimentation,coût total en CAD
84568,Hockeyeur,Végétalien,"$1,000.81"
51085,Hockeyeur,Végétarien,"$70,040.85"
12443,Hockeyeur,Végétarien,"$1,536.96"
60675,Hockeyeur,Végétarien,"$3,105.94"
1095,Hockeyeur,Végétarien,"$22,753.50"
312,Hockeyeur,Végétarien,"$1,157.76"
312,Hockeyeur,Végétarien,"$38,509.17"
18535,Hockeyeur,Végétarien,"$3,857.07"
44485,Hockeyeur,Végétarien,"$1,106.05"
